In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pygad
import wandb

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

tf.random.set_seed(0)
np.random.seed(0)

2023-07-17 13:26:50.800287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 13:26:50.863117: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-17 13:26:52.911202: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/macula/SMATousi/.conda/envs/mac-deep/lib/:/home/macula/SMATousi/.conda/envs/mac-deep/lib/py

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5720365588546116564
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22572105728
locality {
  bus_id: 1
  links {
  }
}
incarnation: 57147650234564996
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2023-07-17 13:26:58.890827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 13:26:58.953787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-17 13:26:58.986714: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-17 13:26:58.986791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at

# Loading the dataset

In [2]:
import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "/home/macula/SMATousi/VAE_outlier/archive/creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)


num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))


counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)
Number of training samples: 227846
Number of validation samples: 56961
Number of positive samples in training data: 417 (0.18% of total)


In [ ]:
train_features[0,:]

# Training the VAE

In [4]:

wandb.init(
    # set the wandb project where this run will be logged
    project="VAE_Outlier_Creditcard_Training_2",
    
    # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 20,
#     }
)

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


latent_dim = 256

encoder_inputs = keras.Input(shape=(30,))
# x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
# x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Flatten()(x)
# x = layers.Dense(128, activation="tanh")(encoder_inputs)
x = layers.Dense(1024, activation="sigmoid")(encoder_inputs)
x = layers.Dense(512, activation="sigmoid")(x)
x = layers.Dense(256, activation="sigmoid")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

latent_inputs = keras.Input(shape=(latent_dim,))
# x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
# x = layers.Reshape((7, 7, 64))(x)
# x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
# decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
x = layers.Dense(256, activation="sigmoid")(latent_inputs)
x = layers.Dense(512, activation="sigmoid")(x)
x = layers.Dense(1024, activation="sigmoid")(x)
# x = layers.Dense(128, activation="tanh")(x)
decoder_outputs = layers.Dense(30, activation="tanh")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

# class CustomCallback(keras.callbacks.Callback):
#     def on_epoch_end(self, batch, logs=None):
#         # Custom logic to execute at the end of each training batch
#         # You can access the training metrics and perform any desired actions
#         # Here is an example of printing the loss at the end of each batch
# #         print(f"Batch {batch}: loss = {logs['loss']}")
# #             wandb.log({"Training/loss": logs["loss"], 
# #            "Training/reconstruction_loss": logs["reconstruction_loss"],
# #            "Training/kl_loss": logs["kl_loss"]})
#         print("#####")
class VAE(keras.Model):
    def __init__(self, encoder, decoder, epochs, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.epochs = epochs
        self.current_epoch = tf.Variable(0, trainable=False, dtype=tf.float32)

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            
            reconstruction_loss = tf.keras.losses.MeanSquaredError()(data, reconstruction)
            
            # Calculate weight using linear decay
            kl_weight = 1.0 - (self.current_epoch / self.epochs)
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
#             kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            kl_loss_weighted = kl_weight * kl_loss
            
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

# Define custom callback to track the current epoch
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.model.current_epoch.assign(epoch)

    def on_epoch_end(self, batch, logs=None):
        # Custom logic to execute at the end of each training batch
        # You can access the training metrics and perform any desired actions
        # Here is an example of printing the loss at the end of each batch
#         print(f"Batch {batch}: loss = {logs['loss']}")
        wandb.log({"Training/loss": logs["loss"], 
       "Training/reconstruction_loss": logs["reconstruction_loss"],
       "Training/kl_loss": logs["kl_loss"]})
#         print("#####")

# Create an instance of VAE model
vae = VAE(encoder, decoder, epochs=100)
vae.compile(optimizer=tf.keras.optimizers.Adam())

# Define the creditdata and custom callback
creditdata = np.concatenate([train_features], axis=0)
creditdata = np.expand_dims(creditdata, -1).astype("float32")
custom_callback = CustomCallback()

# Train the VAE model
vae.fit(creditdata, epochs=100, batch_size=1024, callbacks=[custom_callback])



wandb: Currently logged in as: stmmc (tousi-team). Use `wandb login --relogin` to force relogin


Epoch 1/100
223/223 [==============================] - 3s 7ms/step - loss: 1.0472 - reconstruction_loss: 1.0101 - kl_loss: 0.0023
Epoch 2/100
223/223 [==============================] - 2s 7ms/step - loss: 1.0067 - reconstruction_loss: 1.0036 - kl_loss: 3.1679e-06
Epoch 3/100
223/223 [==============================] - 2s 7ms/step - loss: 0.9985 - reconstruction_loss: 0.9749 - kl_loss: 0.0054
Epoch 4/100
223/223 [==============================] - 2s 7ms/step - loss: 0.9271 - reconstruction_loss: 0.9055 - kl_loss: 0.0166
Epoch 5/100
223/223 [==============================] - 2s 7ms/step - loss: 0.8905 - reconstruction_loss: 0.8537 - kl_loss: 0.0212
Epoch 6/100
223/223 [==============================] - 2s 7ms/step - loss: 0.8438 - reconstruction_loss: 0.7971 - kl_loss: 0.0261
Epoch 7/100
223/223 [==============================] - 1s 6ms/step - loss: 0.7941 - reconstruction_loss: 0.7380 - kl_loss: 0.0321
Epoch 8/100
223/223 [==============================] - 2s 7ms/step - loss: 0.7328 - re

223/223 [==============================] - 2s 7ms/step - loss: 0.4606 - reconstruction_loss: 0.3912 - kl_loss: 0.0604
Epoch 64/100
223/223 [==============================] - 1s 7ms/step - loss: 0.4479 - reconstruction_loss: 0.3913 - kl_loss: 0.0604
Epoch 65/100
223/223 [==============================] - 1s 6ms/step - loss: 0.4543 - reconstruction_loss: 0.3904 - kl_loss: 0.0605
Epoch 66/100
223/223 [==============================] - 2s 7ms/step - loss: 0.4485 - reconstruction_loss: 0.3902 - kl_loss: 0.0604
Epoch 67/100
223/223 [==============================] - 2s 7ms/step - loss: 0.4502 - reconstruction_loss: 0.3895 - kl_loss: 0.0604
Epoch 68/100
223/223 [==============================] - 2s 8ms/step - loss: 0.4391 - reconstruction_loss: 0.3894 - kl_loss: 0.0605
Epoch 69/100
223/223 [==============================] - 2s 7ms/step - loss: 0.4440 - reconstruction_loss: 0.3889 - kl_loss: 0.0604
Epoch 70/100
223/223 [==============================] - 2s 7ms/step - loss: 0.4457 - reconstruct

# Finding the subspaces

In [ ]:
rec_loss = []
detected_outliers = []
inlier_features = []
best_solutions = []

wandb.init(
    # set the wandb project where this run will be logged
    project="VAE_Outlier_GA_Creditcard_subspace_detection_1",
    
    # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 20,
#     }
)

print(" ********** Finding Inliers ************")

inlier_count = 3
c = 0

for j in range(val_features.shape[0]):
    
    test = val_features[j,:]

    test = test.reshape([1,30])

    z_mean, z_log_var, z = vae.encoder(test)
    reconstruction = vae.decoder(z)

    reconstruction_loss = tf.keras.losses.MeanSquaredError()(test,reconstruction)
    reconstruction_loss = reconstruction_loss.numpy()
    
    if reconstruction_loss < 0.4:
        c = c + 1
        inlier_features.append(val_features[j,:])
    
    if c == inlier_count:
        
        break
    
inlier_features = np.array(inlier_features)

print(" ********* Inliers found: The shape: ", inlier_features.shape[0], " **************")
        


for i in range(val_features.shape[0]):
    
    test = val_features[i,:]

    test = test.reshape([1,30])

    z_mean, z_log_var, z = vae.encoder(test)
    reconstruction = vae.decoder(z)

    reconstruction_loss = tf.keras.losses.MeanSquaredError()(test,reconstruction)
    reconstruction_loss = reconstruction_loss.numpy()

    rec_loss.append(reconstruction_loss)
    
    if reconstruction_loss > 3:
        
        print("Rec Loss = ", reconstruction_loss, " -------- Label = ", val_targets[i], " Case No. ", i)
        
        detected_outliers.append([i,val_targets[i]])
        
        def fitness_func_avg_with_penalty(ga_instance, solution, solution_idx):
    
            inliers = inlier_features

            avg_ins = np.mean(inliers, axis=0)
            avg_ins = avg_ins.reshape([1,30])

            particle = val_features[i,:]
            particle = particle.reshape([1,30])

        #     abn_subspace = solution * val_features[6728,:]

        #     abn_subspace = abn_subspace.reshape([1,30])

            avg_in_rec = []

            for index in range(inliers.shape[0]):

                candidate_inlier = inliers[index,:]
                candidate_inlier = candidate_inlier.reshape([1,30])

                in_remain = candidate_inlier * solution

                in_normal_subspace = 1 - solution

                in_replace = in_normal_subspace * avg_ins

                in_candidate = in_remain + in_replace

                z_mean, z_log_var, z = vae.encoder(in_candidate)
                in_candidate_rec = vae.decoder(z)


                rec_loss = tf.keras.losses.MeanSquaredError()(in_candidate,in_candidate_rec)

                avg_in_rec.append(rec_loss.numpy())

            avg_in_rec = np.array(avg_in_rec)
            avg_in_rec = np.mean(avg_in_rec)





        #     z_mean, z_log_var, z = vae.encoder(particle)
        #     reconstruction_1 = vae.decoder(z)

            out_remain = particle * solution

            out_normal_subspace = 1 - solution

            out_replace = avg_ins * out_normal_subspace

            out_candidate = out_remain + out_replace


            z_mean, z_log_var, z = vae.encoder(out_candidate)
            out_candidate_rec = vae.decoder(z)

            rec_loss = tf.keras.losses.MeanSquaredError()(out_candidate,out_candidate_rec)
            rec_loss = rec_loss.numpy()

            # adding the penalty here!

            num_ones = np.count_nonzero(solution == 1)

            penalty_rate = 1.0 / val_features.shape[1]

            regul_param = 1

            penalty_rate = penalty_rate / regul_param

            fitness = rec_loss / (avg_in_rec + num_ones * penalty_rate * avg_in_rec)

    #         fitness = fitness - num_ones * penalty_rate * fitness

            return fitness
        
        
        def on_generation(ga):
            print("Generation", ga.generations_completed)

            solution, solution_fitness, solution_idx = ga_instance.best_solution()

            wandb.log({"GA - " + str(i) + "/solution_fitness": solution_fitness})

            print(solution_fitness)
            
        
        fitness_function = fitness_func_avg_with_penalty

        num_generations = 40
        num_parents_mating = 4

        sol_per_pop = 100
        num_genes = val_features.shape[1]

        init_range_low = -2
        init_range_high = 5

        parent_selection_type = "sss"
        keep_parents = 1

        space = [[0,1] for i in range(num_genes)]

        crossover_type = "single_point"

        mutation_type = "random"
        mutation_percent_genes = 15

        ga_instance = pygad.GA(num_generations=num_generations,
                               num_parents_mating=num_parents_mating,
                               fitness_func=fitness_function,
                               sol_per_pop=sol_per_pop,
                               num_genes=num_genes,
                               init_range_low=init_range_low,
                               init_range_high=init_range_high,
                               parent_selection_type=parent_selection_type,
    #                            keep_parents=keep_parents,
                               keep_elitism=2,
                               crossover_type=crossover_type,
                               mutation_type=mutation_type,
                               mutation_percent_genes=mutation_percent_genes,
                               on_generation=on_generation,
                               gene_space = space)

        ga_instance.run()

        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        print("##########  End of the outlier numver ", i, " epoch ##########")
        print("Parameters of the best solution : {solution}".format(solution=solution))

        best_solutions.append(solution)

        wandb.log({"Solutions/solution": solution})

        


detected_outliers = np.array(detected_outliers)
np.save("creditcard_detected_outliers_1.npy", detected_outliers)

best_solutions = np.array(best_solutions)
np.save("creditcard_outlier_subspaces_1.npy", best_solutions)


rec_loss = np.array(rec_loss)
print("mean = ", np.mean(rec_loss))
print("std = ", np.std(rec_loss))

In [ ]:
rec_loss = []

for i in range(train_features.shape[0]):
        
    if train_targets[i] == 1:
        test = train_features[i,:]

        test = test.reshape([1,30])

        z_mean, z_log_var, z = vae.encoder(test)
        reconstruction = vae.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(test,reconstruction)



    #     if reconstruction_loss.numpy() > 40:

        rec_loss.append(reconstruction_loss.numpy())
        print("Rec Loss = ", reconstruction_loss.numpy(), " -------- Label = ", train_targets[i], " Case No. ", i)

rec_loss = np.array(rec_loss)
print("mean = ", np.mean(rec_loss))
print("std = ", np.std(rec_loss))